In [53]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.cross_validation import *
from sklearn.grid_search import GridSearchCV

/home/avens8/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%%time
store = pd.HDFStore("../../data.h5")
keys = store.keys()
print ("there are {} rows in a table".format(len(keys)))
EPS = 0.01

there are 10542 rows in a table
CPU times: user 19.2 s, sys: 528 ms, total: 19.7 s
Wall time: 19.7 s


In [10]:
data = pd.SparseDataFrame()

In [11]:
for i in tqdm(keys):
    if i == '/denpr_id' or i == '/denimg_num':
        continue
    columns = store[i] < EPS
    row = store[i]
    columns_name = keys[2]
    row.loc[columns] = 0
    #print('{} not Nan for key {}'.format(row.shape[0] - row.isnull().sum(), i))
    sparse_row = pd.SparseSeries(data=row, fill_value=0)
    data[i] = sparse_row


  2%|▏         | 233/10542 [00:06<04:39, 36.82it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/avens8/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/avens8/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/avens8/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 10542/10542 [15:01<00:00, 11.69it/s]


In [18]:
target = pd.read_csv('../../val.csv')

In [44]:
y = pd.DataFrame()
y['denpr_id'] = target[target['train'] == 0]['id']
y['cat'] = target[target['train'] == 0]['cat']

In [47]:
y.sort_values('denpr_id', inplace=True)
y.reset_index(drop=True, inplace=True)
y.drop(['denpr_id'], axis=1, inplace=True)
y.shape

(494555, 1)

In [55]:
data.drop(['/denpr_id', '/denimg_num'], axis=1, inplace=True)

In [58]:
features = data.columns
data['cat_id'] = y['cat']

In [63]:
features = list(features)

In [90]:
xgb_model = xgb.XGBClassifier(
        nthread = 8,
        silent = 1,
        )
parameters = {
    'n_estimators': [250, 500, 1000],
    'learning_rate': [0.05, 0.1, 0.3],
    'max_depth': [6, 9, 12],
}

In [92]:
clf = GridSearchCV(xgb_model, parameters, n_jobs=1, 
                   cv=StratifiedKFold(data['cat_id'], n_folds=3, shuffle=True), 
                   scoring='roc_auc',
                   verbose=2, refit=True)

/home/avens8/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=2.
  % (min_labels, self.n_folds)), Warning)
Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7ff03b835d30>>
Traceback (most recent call last):
  File "/home/avens8/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 324, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'


In [93]:
clf.fit(data[features], pd.Series(data['cat_id']))

Fitting 2 folds for each of 27 candidates, totalling 54 fits
[CV] learning_rate=0.05, max_depth=6, n_estimators=250 ...............


MemoryError: 

In [83]:
features[5268: 5273]

['/den998', '/den999', '/res1', '/res10', '/res100']

In [73]:
types

[]

In [ ]:
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

test_probs = clf.predict_proba(test[features])[:,1]

sample = pd.read_csv('../input/sample_submission.csv')
sample.QuoteConversion_Flag = test_probs
sample.to_csv("xgboost_best_parameter_submission.csv", index=False)